In [2]:
!pip3 install pprint
from pprint import pprint

Could not build wheels for pprint, since package 'wheel' is not installed.


In [3]:
!pip3 install beautifulsoup4

Could not build wheels for beautifulsoup4, since package 'wheel' is not installed.
Could not build wheels for soupsieve, since package 'wheel' is not installed.


In [4]:
!pip3 install --upgrade pip

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (20.1)
Could not build wheels for pip, since package 'wheel' is not installed.


In [5]:
!pip3 install requests

Could not build wheels for requests, since package 'wheel' is not installed.
Could not build wheels for idna, since package 'wheel' is not installed.
Could not build wheels for certifi, since package 'wheel' is not installed.
Could not build wheels for chardet, since package 'wheel' is not installed.
Could not build wheels for urllib3, since package 'wheel' is not installed.


In [6]:
from six.moves.urllib.parse import quote as _quote
from bs4 import BeautifulSoup as _BeautifulSoup
import requests as _requests
import urllib
import os
import pandas as pd

__BASE_URL__ = 'https://lyrics.wikia.com'

In [7]:
class LyricsNotFound(Exception):
    __module__ = Exception.__module__

    def __init__(self, message=None):
        super(LyricsNotFound, self).__init__(message)

In [8]:
class LanguageNotFound(Exception):
    __module__ = Exception.__module__

    def __init__(self, message=None):
        super(LanguageNotFound, self).__init__(message)

In [9]:
def urlize(string):
    """Convert string to LyricWikia format"""
    return _quote('_'.join(string.title().split()))

In [10]:
def create_url(artist, song):
    """Create the URL in the LyricWikia format"""
    return (__BASE_URL__ +
            '/wiki/{artist}:{song}'.format(artist=urlize(artist),
                                           song=urlize(song)))

In [11]:
def get_lyrics(artist, song, linesep='\n', timeout=None):
    """Retrieve the lyrics of the song and return the first one in case
    multiple versions are available."""
    return get_all_lyrics(artist, song, linesep, timeout)[0]

In [12]:

def get_all_lyrics(artist, song, linesep=' \n ', timeout=None):
    """Retrieve a list of all the lyrics versions of a song."""
    url = create_url(artist, song)
    response = _requests.get(url, timeout=timeout)
    soup = _BeautifulSoup(response.content, "html.parser")
    lyricboxes = soup.findAll('div', {'class': 'lyricbox'})

    if not lyricboxes:
        raise LyricsNotFound('Cannot download lyrics')

    for lyricbox in lyricboxes:
        for br in lyricbox.findAll('br'):
            br.replace_with(linesep)

    return [lyricbox.text.strip() for lyricbox in lyricboxes]

In [22]:
def get_songs_by_language(language, linesep=' \n ', timeout=None):
    """Retrieve a dataframe of all lyrics versions of a song."""
    df = pd.DataFrame(columns=['Artist', 'Title'])
    url = "https://lyrics.fandom.com/wiki/Category:Language/"+language
    df = parse_page_now(url,df)
    return df

In [28]:
def parse_page_now(url, df, timeout=None):
    response = _requests.get(url, timeout=timeout)
    soup = _BeautifulSoup(response.content, "html.parser")
    data = soup.findAll('li',attrs={'class':'category-page__member'})
    
    if not data:
        raise LanguageNotFound('No such language')
    
    for div in data:
        links = div.findAll('a')
        for a in links:
            
            lyric_link = a['href'].strip('/wiki/')
            artist = lyric_link.split(":")[0]
            title = lyric_link.split(":")[1]
            if(artist == "Category"):
                continue
            df = df.append({'Artist': artist, 'Title': title}, ignore_index=True)
    
    if(soup.find('div', attrs={'class':'category-page__pagination'}) == None):
        return df

    next_page_text = soup.find('div', attrs={'class':'category-page__pagination'}).find('a', attrs={'class':'category-page__pagination-next wds-button wds-is-secondary'})
    if next_page_text != None:
        next_page_url = next_page_text['href']
        df = parse_page_now(next_page_url,df)
    return df

In [15]:
class Song(object):
    """A Song backed by the LyricWikia API"""

    def __init__(self, artist, title):
        self.artist = artist
        self.title = title

    @property
    def lyrics(self):
        """Song lyrics obtained by parsing the LyricWikia page"""
        return get_lyrics(self.artist, self.title)

    def __str__(self):
        return "Song(artist='%s', title='%s')" % (self.artist, self.title)

    def __repr__(self):
        return str(self)

In [16]:
class Album(object):
    """An Album backed by the LyricWikia API"""

    def __init__(self, artist, album_data):
        self.artist = artist
        self.title = album_data['album']
        self.year = album_data['year']
        self.songs = [Song(artist, song) for song in album_data['songs']]

    def __str__(self):
        return "Album(artist='%s', title='%s')" % (self.artist, self.title)

    def __repr__(self):
        return str(self)

In [17]:

class Artist(object):
    """An Artist backed by the LyricWikia API"""

    __API__ = __BASE_URL__ + '/api.php?fmt=json&func=getArtist&artist={artist}'

    def __init__(self, name):
        url = self.__API__.format(artist=urlize(name))
        data = _requests.get(url).json()
        self.name = data['artist']
        self.albums = [Album(self.name, album) for album in data['albums']]

    def __str__(self):
        return "Artist(name='%s')" % (self.name)

    def __repr__(self):
        return str(self)

In [29]:
get_songs_by_language('Gujarati')

,Artist,Title
0,Mondstille,In_Mondes_Stille
